In [17]:
#SINCE NOW ALL FILES ARE IN THE SEPARATE FUNCTIONS FOLDER, ALL PATH WERE CHANGED SO THAT THEY GO 1 FOLDER UP.
#example: '../derivatives/sub' instead of './derivatives/sub'


#Load data, filter, make folders
import numpy as np
import plotly as plotly
import plotly.graph_objects as go
import pandas as pd
import chart_studio.plotly as py

from main_meg_qc import initial_stuff


In [18]:

n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff()

Opening raw data file ../data/sub_HT05ND16/210811/mikado-1.fif/...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/../data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif :

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:8: RuntimeWarning:

This filename (../data/sub_HT05ND16/210811/mikado-1.fif/) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz



Reading 0 ... 300000  =      0.000 ...   300.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 100.00 Hz: -6.02, -6.02 dB

Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
301 events found
Event IDs: [    9    19    21    23    27    31 16393 16402 16403 16405 16411 32741
 32746 32749 32750 32759]


/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:64: RuntimeWarning:

Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug



Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
253 events found
Event IDs: [    9    19    20    21    22    23    27    31 16393 16402 16403 16405
 16411 32741 32746 32749 32750 32759]
Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
8 events found
Event IDs: [ 9 19 20 21 22]
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped


/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:64: RuntimeWarning:

Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:79: RuntimeWarning:

Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug



In [19]:
#%% RMSE - general function to use in other functions
#STD CALCULATION IS MUCH LESS COdE BUT TAKES LONGER THAN RMSE

def RMSE(data_m_or_g: np.array or list) -> np.array:

    data_m_or_g=np.array(data_m_or_g) #convert to numpy array if it s not
    rmse_list=[]

    data_minentions=len(data_m_or_g.shape)
    if data_minentions==2: #if the data has raws and columns - iterate over raws. if input is 1 dimentional - juat calculate the whole thing.
        for i, dat_raw in enumerate (data_m_or_g):
            y_actual=dat_raw
            y_pred=y_actual.mean()
            rmse_data=np.sqrt(((y_pred - y_actual) ** 2).mean())
            rmse_list.append(rmse_data)
    elif data_minentions==1:
        y_actual=data_m_or_g
        y_pred=data_m_or_g.mean()
        rmse_data=np.sqrt(((y_pred - y_actual) ** 2).mean())
        rmse_list.append(rmse_data)
    else:
        print('Only 1 or 2 dimentional data is accepted, not more!')
        return

    rmse_nd=np.array(rmse_list) #conver to numpy array

    return rmse_nd

In [20]:
selected_mags = [item[1] for item in mags]
selected_grads = [item[1] for item in grads]
data_mags, times = filtered_d_resamp[selected_mags, :]  
data_grads, times = filtered_d_resamp[selected_grads, :]  

In [21]:
# Root mean squared error calculation (or STD - same result) over ALL data (not epoched):

def RMSE_meg_all(data, mags: list, grads: list, std_lvl: int): #, min_duration_event=1, epoch_tmin=-0.2, epoch_tmax=1):
    #give path to directory and then it should auto find the data file when bids compliant.
    ##maybe distinguish between negative and positive std_lvl

    #sid - subject Id. Has to be a string , like '1'
    #mags, grads - channels like [name, index]
  

    # Separate data for mags and grads in 2 arrays.
    selected_mags = [item[1] for item in mags]
    selected_grads = [item[1] for item in grads]
    data_mags, times = data[selected_mags, :]  
    data_grads, times = data[selected_grads, :]  

    # %% Calculate STD or RMSE of each channel

    #Calculate RMSE for each channel (separated mags and grads) - for the entire time duration:
    std_mags = RMSE(data_mags)
    std_grads = RMSE(data_grads)

    #STD (if wanna use insted of RMSE. it will exactly replace the RMSE function above):
    #std_mags=np.std(data_mags, axis=1) #calculate std of all magnetometers (along second dimantion)
    #std_grads=np.std(data_grads, axis=1) #calculate std of all gradiometers (along second dimantion)


    # Check if channel data is within 1 std over all channels.
    # COMMENT: can use -3 to 3 (or other number) std istead of -1/+1 std, but this can adjusted later. 
    # 1 std is too narrow, gives way too many bad channels.

    std_std_mags=np.std(std_mags)
    std_std_grads=np.std(std_grads)

    mean_std_mags=np.mean(std_mags)
    mean_std_grads=np.mean(std_grads)

    ch_ind_large_std_mags= np.where(std_mags > mean_std_mags+std_lvl*std_std_mags) #find magn channels with largest std
    ch_ind_large_std_grads= np.where(std_grads > mean_std_grads+std_lvl*std_std_grads) 
    ch_ind_small_std_mags= np.where(std_mags < mean_std_mags-std_lvl*std_std_mags) #find magn channels with smallest std
    ch_ind_small_std_grads= np.where(std_grads < mean_std_grads-std_lvl*std_std_grads)

    magn_channel_big_std=np.array(mags)[ch_ind_large_std_mags] #find the name of the magn with largest std 
    grad_channel_big_std=np.array(grads)[ch_ind_large_std_grads]
    magn_channel_small_std=np.array(mags)[ch_ind_small_std_mags]
    grad_channel_small_std=np.array(grads)[ch_ind_small_std_grads]


    #This function simply makes a list of tuples. Each tuple is: name of channel, std value.
    #Each tuple represents channel with too big or too small std, calculated over whole data.

    def Channels_with_nonnormal_stds(ch_ind, all_stds_m_or_g, channels_big_std_names):
        channel_big_std_vals=all_stds_m_or_g[ch_ind]
        nonnormal_std_with_value=[]
        for id, val in enumerate (ch_ind[0]):
            new_tuple=(channels_big_std_names[id][0],  channel_big_std_vals[id])
            nonnormal_std_with_value.append(new_tuple)
        return(nonnormal_std_with_value)

    #Apply function above:
    m_big_std_with_value=Channels_with_nonnormal_stds(ch_ind_large_std_mags, std_mags, magn_channel_big_std)
    g_big_std_with_value=Channels_with_nonnormal_stds(ch_ind_large_std_grads, std_grads, grad_channel_big_std)
    m_small_std_with_value=Channels_with_nonnormal_stds(ch_ind_small_std_mags, std_mags, magn_channel_small_std)
    g_small_std_with_value=Channels_with_nonnormal_stds(ch_ind_small_std_grads, std_grads, grad_channel_small_std)
        
    #Return the channel names with STD over the set STD level and under the set negative STD level.
    return(m_big_std_with_value, g_big_std_with_value, m_small_std_with_value, g_small_std_with_value, std_mags, std_grads)


In [22]:
#Try: USING RESAMPLED DATA HERE

m_big_std_with_value, g_big_std_with_value, m_small_std_with_value, g_small_std_with_value, rmse_mags, rmse_grads=RMSE_meg_all(data=filtered_d_resamp, 
    mags=mags, grads=grads, std_lvl=1)

print('Magnetometers with high std:')
print(m_big_std_with_value)

print('Gradiometers with high std:')
print(g_big_std_with_value)

Magnetometers with high std:
[('MEG0621', 4.965338372879145e-13), ('MEG1021', 4.852992071039414e-13), ('MEG1031', 5.004345361972638e-13), ('MEG1041', 4.939759752048587e-13), ('MEG1421', 5.678604995893133e-13), ('MEG1431', 5.476084157649638e-13), ('MEG1741', 5.282172136198286e-13), ('MEG2121', 5.018134686600477e-13), ('MEG2131', 5.860882537731944e-13), ('MEG2141', 5.460444751629966e-13), ('MEG2331', 5.002680469283238e-13), ('MEG2531', 5.683082618711569e-13), ('MEG2541', 5.925124304516582e-13), ('MEG2621', 5.364787072987134e-13), ('MEG2631', 5.466366791474147e-13)]
Gradiometers with high std:
[('MEG1023', 7.179247707983359e-12), ('MEG1323', 6.757557360268088e-12), ('MEG1412', 6.777417001822614e-12), ('MEG1442', 6.795936392344075e-12), ('MEG1713', 7.471548646598566e-12), ('MEG1742', 7.777625918922468e-12), ('MEG1743', 8.688099976270316e-12), ('MEG1813', 7.323041849074902e-12), ('MEG1922', 7.254700915844952e-12), ('MEG1923', 6.853831580367702e-12), ('MEG1932', 6.896523110045817e-12), ('MEG

In [23]:
def boxplot_std_hovering_plotly(std_data: list, tit: str, channel_names: list, sid: str):

  #Using plotly! https://plotly.github.io/plotly.py-docs/generated/plotly.graph_objects.Box.html

  unit='?'
  if tit=='Magnetometers':
    unit='T'
  elif tit=='Gradiometers':
    unit='T/m'
  else:
    print('Please check tit input. Has to be "Magnetoneters" or "Gradiometers"')

  ch_names=[m[0] for m in channel_names] #names of channels for annotating the plot
  df = pd.DataFrame (std_data, index=ch_names, columns=['std'])

  fig = go.Figure()

  fig.add_trace(go.Box(x=df['std'],
  name="",
  text=df['std'].index, 
  opacity=0.7, 
  boxpoints="all", 
  pointpos=0,
  marker_size=5,
  line_width=1))
  fig.update_traces(hovertemplate='%{text}'+'<br>STD: %{x: .0f}')
      

  fig.update_layout(
      yaxis={'visible': False, 'showticklabels': False},
      xaxis = dict(
      showexponent = 'all',
      exponentformat = 'e'),
      xaxis_title="standard deviation in "+unit,
      title={
      'text': "Standard deviation of the data for "+tit,
      'y':0.85,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'})
      
  fig.show()

  fig_name='Stds_all_data_'+tit+'.html'
  fig_path='../derivatives/sub-'+sid+'/megqc/figures/'+fig_name
  fig.write_html(fig_path)

  return(fig, fig_path)

In [24]:
fig_m, fig_path_m=boxplot_std_hovering_plotly(std_data=rmse_grads, tit='Gradiometers', channel_names=grads, sid='1')



In [25]:
#Create a function which will loop over mags or grads (separate epochs) and calculate std (used in RMSE_meg_epoch):
def std_mg(mg_names: list, df_mg: pd.DataFrame, epoch_numbers: list) -> pd.DataFrame:

    import pandas as pd
    
    dict_mg = {}

    for ep in epoch_numbers: #loop over each epoch
        rows_for_ep = [row for row in df_mg.iloc if row.epoch == ep] #take all rows of 1 epoch, all channels.
        #std_epoch = [] #list with stds
        rmse_epoch=[]

        for ch_name in mg_names: #loop over channel names
            data_ch_epoch = [row_mg[ch_name] for row_mg in rows_for_ep] #take the data for 1 epoch for 1 channel
            rmse_ch_ep = RMSE(data_ch_epoch)
            rmse_ch_ep=np.float64(rmse_ch_ep) #convert from ndarray to float
            rmse_epoch.append(rmse_ch_ep)

            #std_ch_ep = np.std(data_ch_epoch) #if want to use std instead
            

        dict_mg[ep] = rmse_epoch

    df_std_mg = pd.DataFrame(dict_mg, index=mg_names)

    return(df_std_mg)

In [26]:
# STD over epochs: use 2 separate data frames for mags and grads in calculations:

def RMSE_meg_epoch(mags: list, grads: list, std_lvl: int, n_events: int, df_epochs_mags: pd.DataFrame, df_epochs_grads:pd.DataFrame, sid: str):

#def RMSE_meg_epoch(data=None, std_lvl=1, stim_channel='STI101',  min_duration_event=1.2, epoch_tmin=-0.2, epoch_tmax=1):
    #stim_channel name is users input. but we can also let mne find it itself if 'None' is set? mne seems to have such function.

    # 1) Loop over the epochs of each channel and check for every separate magn and grad and calculate std
    
    eps=list(range(0,n_events)) #list of epoch numbers
   
    mags_names = [mag[0] for mag in mags]
    grads_names = [grad[0] for grad in grads]


    #Apply function from above for mags and grads:
    df_std_mags=std_mg(epoch_numbers=eps, df_mg=df_epochs_mags, mg_names=mags_names)
    df_std_grads=std_mg(epoch_numbers=eps, df_mg=df_epochs_grads, mg_names=grads_names)

    # 2) Check (which epochs for which channel) are over 1STD (or 2, 3, ets STDs) for (this epoch for all channels)

    #Find what is 1 std over all channels per 1 epoch:
    std_std_mags_per_epoch=[]
    std_std_grads_per_epoch=[]
    mean_std_mags_per_epoch=[]
    mean_std_grads_per_epoch=[]

    for ep in eps: #goes over each epoch
        std_std_mags_per_epoch.append(np.std(df_std_mags.iloc[:, ep])) #std of stds of all channels of every single epoch
        std_std_grads_per_epoch.append(np.std(df_std_grads.iloc[:, ep]))

        mean_std_mags_per_epoch.append(np.mean(df_std_mags.iloc[:, ep])) #mean of stds of all channels of every single epoch
        mean_std_grads_per_epoch.append(np.mean(df_std_grads.iloc[:, ep]))


    df_ch_ep_large_std_mags=df_std_mags.copy()
    df_ch_ep_large_std_grads=df_std_grads.copy()

    df_ch_ep_small_std_mags=df_std_mags.copy()
    df_ch_ep_small_std_grads=df_std_grads.copy()

    #Now see which channles in epoch are over 1 std or under -1 std:
    for ep in eps: #goes over each epoch   
        df_ch_ep_large_std_mags.iloc[:,ep] = df_ch_ep_large_std_mags.iloc[:,ep] > mean_std_mags_per_epoch[ep]+std_lvl*std_std_mags_per_epoch[ep] #magnetometers
        df_ch_ep_large_std_grads.iloc[:,ep] = df_ch_ep_large_std_grads.iloc[:,ep] > mean_std_grads_per_epoch[ep]+std_lvl*std_std_grads_per_epoch[ep] #gradiometers

        df_ch_ep_small_std_mags.iloc[:,ep] = df_ch_ep_small_std_mags.iloc[:,ep] < mean_std_mags_per_epoch[ep]-std_lvl*std_std_mags_per_epoch[ep] #magnetometers
        df_ch_ep_small_std_grads.iloc[:,ep] = df_ch_ep_small_std_grads.iloc[:,ep] < mean_std_grads_per_epoch[ep]-std_lvl*std_std_grads_per_epoch[ep] #gradiometers


    # Create csv files  for the user:

    df_std_mags.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/std_mags_per_epoch.csv')
    
    df_std_grads.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/std_grads_per_epoch.csv')

    df_ch_ep_large_std_mags.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Large_std_mags_per_epoch.csv')

    df_ch_ep_large_std_grads.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Large_std_grads_per_epoch.csv')

    df_ch_ep_small_std_mags.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Small_std_mags_per_epoch.csv')

    df_ch_ep_small_std_grads.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Small_std_grads_per_epoch.csv')

    return(df_std_mags, df_std_grads)

In [29]:
#try (will output csv files):
# USING NON RESAMPLED DATA

df_std_mags, df_std_grads=RMSE_meg_epoch(mags=mags, grads=grads, std_lvl=1, n_events=n_events, df_epochs_mags=df_epochs_mags, df_epochs_grads=df_epochs_grads, sid='1') 

In [30]:
def boxplot_std_channel_epoch_hovering_plotly(df_mg: pd.DataFrame, tit: str, sid: str):

  unit='?'
  if tit=='Magnetometers':
    unit='T'
  elif tit=='Gradiometers':
    unit='T/m'
  else:
    print('Please check tit input. Has to be "Magnetoneters" or "Gradiometers"')

  #transpose the data to plot the boxplots on x axes
  df_mg_transposed = df_mg.T 

  #collect all names of original df into a list to use as tick labels:
  ch_names=list(df_mg_transposed) 

  fig = go.Figure()

  for col in df_mg_transposed:
    fig.add_trace(go.Box(y=df_mg_transposed[col].values, 
      name=df_mg_transposed[col].name, 
      opacity=0.7, 
      boxpoints="all", 
      pointpos=0,
      marker_size=3,
      line_width=1,
      text=df_mg_transposed[col].index))
    fig.update_traces(hovertemplate='Epoch: %{text}'+'<br>STD: %{y: .0f}')
      

  fig.update_layout(
      xaxis = dict(
          tickmode = 'array',
          tickvals = [v for v in range(0, len(ch_names))],
          ticktext = ch_names,
          rangeslider=dict(visible=True)
      ),
      yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'),
      yaxis_title="standard deviation in "+unit,
      title={
        'text': "STDs over epochs for "+tit,
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
    
  fig.show()

  fig_name='Stds_epochs_per_channel_'+tit+'.html'
  fig_path='../derivatives/sub-'+sid+'/megqc/figures/'+fig_name
  fig.write_html(fig_path)

  return(fig, fig_path)

  #figure out weird y axis, hovering is too long, simple slider

In [31]:
fig_mags_std_epoch, fig_path_std_epoch=boxplot_std_channel_epoch_hovering_plotly(df_mg=df_std_mags, tit='Magnetometers', sid='1')

In [51]:
# Create an html report:
# The actual plotly tutorial was outdated. 
# better version here: https://towardsdatascience.com/automated-interactive-reports-with-plotly-and-python-88dbe3aae5

sid='1'

with open(fig_path_m, 'r') as f1:
    fig1 = f1.read()
    
with open(fig_path_std_epoch, 'r') as f2:
    fig2 = f2.read()
    
html_string = '''
<!doctype html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>MEG QC: RMSE Report</title>
        <style>body{ margin:0 100;}</style>
    </head>
    
    <body style="font-family: Arial">
        <center>
        <h1>MEG data quality analysis report</h1>
        <br></br>
        <!-- *** Section 1 *** --->
        <h2>Standard deviation over the entire data</h2>
        ''' + fig1 + '''
        <p>graph description...</p>
         
        <!-- *** Section 2 *** --->
        <br></br>
        <br></br>
        <br></br>
        <h2>Standard deviation over epochs</h2>
        ''' + fig2 + '''
        <p>graph description...</p>
        </center>
    
    </body>
</html>'''

with open('../derivatives/sub-'+sid+'/megqc/figures/report_RMSE.html', 'w', encoding = 'utf8') as f:
    f.write(html_string)